# Routes

In [ ]:
from fasthtml.common import *

In [ ]:
app = FastHTML()

Behaviour in FastHTML apps is defined by routes. The syntax is largely the same as the wonderful [FastAPI](https://fastapi.tiangolo.com/) (which is what you should be using instead of this if you're creating a JSON service. FastHTML is for mainly for making HTML web apps, not APIs).

Note that you need to include the types of your parameters, so that `FastHTML` knows what to pass to your function. Here, we're just expecting a string:

In [ ]:
@app.get('/user/{nm}')
def get_nm(nm:str): return f"Good day to you, {nm}!"

Normally you'd save this into a file such as main.py, and then run it in `uvicorn` using:

```
uvicorn main:app
```

However, for testing, we can use Starlette's `TestClient` to try it out:

In [ ]:
from starlette.testclient import TestClient

In [ ]:
client = TestClient(app)
r = client.get('/user/Jeremy')
r

<Response [200 OK]>

TestClient uses `httpx` behind the scenes, so it returns a `httpx.Response`, which has a `text` attribute with our response body:

In [ ]:
r.text

'Good day to you, Jeremy!'

In the previous example, the function name (`get_nm`) didn't actually matter -- we could have just called it `_`, for instance, since we never actually call it directly. It's just called through HTTP. In fact, we often do call our functions `_` when using this style of route, since that's one less thing we have to worry about naming.

An alternative approach to creating a route is to use `app.route` instead, in which case you make the function name the HTTP method you want. Since this is such a common pattern, you might like to give a shorter name to `app.route` -- we normally use `rt`:

In [ ]:
rt = app.route

@rt('/')
def post(): return "Going postal!"

client.post('/').text

'Going postal!'

## Features

Here's a brief demo of all the features of the library:

In [ ]:
from starlette.responses import Response
from datetime import datetime
from fastcore.utils import *
from dataclasses import dataclass, asdict
from IPython import display

In [ ]:
def todict(req): return {k:str(v) for k,v in req.items()}

In [ ]:
app = FastHTML()
rt = app.route

@app.get("/")
def _(req): return todict(req.scope)

In [ ]:
cli = TestClient(app)
r = cli.get('/')
print(r.text)

{"type":"http","http_version":"1.1","method":"GET","path":"/","raw_path":"b'/'","root_path":"","scheme":"http","query_string":"b''","headers":"[(b'host', b'testserver'), (b'accept', b'*/*'), (b'accept-encoding', b'gzip, deflate, br'), (b'connection', b'keep-alive'), (b'user-agent', b'testclient')]","client":"['testclient', 50000]","server":"['testserver', 80]","extensions":"{'http.response.debug': {}}","state":"{}","app":"<fasthtml.core.FastHTML object>","session":"{}","starlette.exception_handlers":"({<class 'starlette.exceptions.HTTPException'>: <bound method ExceptionMiddleware.http_exception of <starlette.middleware.exceptions.ExceptionMiddleware object>>, <class 'starlette.exceptions.WebSocketException'>: <bound method ExceptionMiddleware.websocket_exception of <starlette.middleware.exceptions.ExceptionMiddleware object>>}, {})","router":"<fasthtml.core.RouterX object>","endpoint":"<function _wrap_ep.<locals>._f>","path_params":"{}"}


In [ ]:
@app.get('/user/{nm}')
def _(nm:str): return f"Good day to you, {nm}!"

cli.get('/user/jph').text

'Good day to you, jph!'

In [ ]:
@rt('/html/{idx}')
async def get(idx:int):
    return Body(
        H4("Wow look here"),
        P(f'It looks like you are visitor {idx}! Next is {idx+1}.')
    )

display.HTML(cli.get('/html/1').text)

In [ ]:
reg_re_param("imgext", "ico|gif|jpg|jpeg|webm")

@app.get(r'/static/{path:path}{fn}.{ext:imgext}')
def get_img(fn:str, path:str, ext:str): return f"Getting {fn}.{ext} from /{path}"

cli.get('/static/foo/jph.ico').text

'Getting jph.ico from /foo/'

In [ ]:
ModelName = str_enum('ModelName', "alexnet", "resnet", "lenet")

@app.get("/models/{nm}")
def model(nm:ModelName): return nm

print(cli.get('/models/alexnet').text)

alexnet


In [ ]:
@app.get("/files/{path}")
async def txt(path: Path): return path.with_suffix('.txt')

print(cli.get('/files/foo').text)

foo.txt


In [ ]:
fake_db = [{"name": "Foo"}, {"name": "Bar"}]

@app.get("/items/")
def read_item(idx:int|None = 0): return fake_db[idx]

print(cli.get('/items/?idx=1').text)

{"name":"Bar"}


In [ ]:
print(cli.get('/items/').text)

{"name":"Foo"}


In [ ]:
@app.get("/booly/")
def booly(coming:bool=True): return 'Coming' if coming else 'Not coming'

print(cli.get('/booly/?coming=true').text)

Coming


In [ ]:
print(cli.get('/booly/?coming=no').text)

Not coming


In [ ]:
@app.get("/datie/")
def datie(d:date): return d

date_str = "17th of May, 2024, 2p"
print(cli.get(f'/datie/?d={date_str}').text)

2024-05-17 14:00:00


In [ ]:
@dataclass
class Bodie:
    a:int;b:str

@rt("/bodie/{nm}")
async def post(nm:str, data:Bodie):
    res = asdict(data)
    res['nm'] = nm
    return res

cli.post('/bodie/me', data=dict(a=1, b='foo')).text

'{"a":1,"b":"foo","nm":"me"}'

In [ ]:
@app.get("/setcookie")
async def setc(req):
    now = datetime.now()
    res = Response(f'Set to {now}')
    res.set_cookie('now', str(now))
    return res

cli.get('/setcookie').text

'Set to 2024-07-26 15:52:40.314109'

In [ ]:
@app.get("/getcookie")
async def getc(now:date): return f'Cookie was set at time {now.time()}'

cli.get('/getcookie').text

'Cookie was set at time 15:52:40.314109'

In [ ]:
@app.get("/ua")
async def ua(user_agent:str): return user_agent

cli.get('/ua', headers={'User-Agent':'FastHTML'}).text

'FastHTML'

In [ ]:
@app.get("/hxtest")
def hxtest(htmx): return htmx.request

cli.get('/hxtest', headers={'HX-Request':'1'}).text

'1'